# Ground magnetic perturbation forecasting

A full-earth ground magnetic perturbation forecasting model using deep learning

In this notebook, we shall see how to forecast ground magnetic perturbation across the northern hemisphere using solar wind measurements.

Authors: Vishal Upendran, Panos Tigas, Bashi Ferdousi, Teo Bloch, Mark Cheung, Siddha Ganju, Asti Bhat, Ryan McGranaghan, Yarin Gal


**Cite as:** Tigas, P., Bloch, T., Upendran, V., Ferdoushi, B., Cheung, M., Ganju, S., McGranaghan, R.M., Gal, Y. and Bhatt, A., 2021. Global Earth Magnetic Field Modeling and Forecasting with Spherical Harmonics Decomposition. arXiv preprint [arXiv:2102.01447](https://arxiv.org/abs/2102.01447).

## Introduction

Ground magnetic field perturbation is currently estimated based on Magnetic Hydrodynamic (MHD) and empirical models. While useful, MHD models are computationally expensive for high-resolution models that are required for small-scale perturbation, and empirical models do not provide a dynamic forecast. 

In this study, we use Spherical Harmonics (SH) to create high-resolution global models of the northward ($\delta\mathrm{B}_\mathrm{N}$) and Eastward ($\delta\mathrm{B}_\mathrm{E}$) components of the perturbation with a lead time of 30 min. How do we do this?

We summarize 2-hours of solar wind conditions measured at L1 point (from the OMNI dataset) using Recurrent Neural Networks (RNNs) to generate a summary "hidden state" of the solar wind measurements. This state is then passed to a fully-connected layer of Multi Layer Perceptro (MLP) to generate a vector of coefficients. These coefficients are then contracted with a basis of spherical harmonics which is generated every forward pass depending on the locations of the different stations which have measurements. The output after the contraction operation is a set of ($\delta\mathrm{B}_\mathrm{N}$) and ($\delta\mathrm{B}_\mathrm{E}$) measurements at each station location.

In this notebook, we shall generate the forecast for the 2015 storm dataset, and visualize the global predictions with the targets as a time series.

## Table of contents

1. Notebook setup
2. Download and read data
3. Setup model
4. Generate forecast
5. Time series of forecast

## Notebook setup

We first need to clone the repo and install packages. Then, we will need to import them.

You will mostly be asked to restart runtime after installation. In such a case, you don't need to run this cell after the packages have been installed.

In [ ]:
!git clone https://github.com/ptigas/geoeffectivenet.git
%cd geoeffectivenet/
!pip install -r requirements.txt

In [ ]:
!pip install torchtext==0.10.0 torch==1.9.0 pytorch-lightning==1.3.5

⚠️ Restart the runtime, and run from the cell labelled "cell 3" below ⚠️ 

In [ ]:
# cell 3

if torch.cuda.is_available() == False:
    print(
        f'CUDA is unavailable. If you are running this notebook on Colab, go to Runtime > Change runtime type, and set "GPU"'
    )
else:
    print("CUDA is available")

In [1]:
%cd geoeffectivenet/

[Errno 2] No such file or directory: 'geoeffectivenet/'
/Users/pwright/Documents/personal/SpaceML/helionb-geoeff/notebooks/01_geoeff_2020


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
import h5py
from astropy.time import Time
import numpy as np
import glob
import pandas as pd
import re
from datetime import datetime
import torch
from torch.utils import data
from matplotlib import cm

In [ ]:
import matplotlib as mpl

## Helper functions 

Define some important helper functions to be used later.

In [ ]:
import sys
from dataloader import SuperMAGIAGADataset,OMNIDataset,ShpericalHarmonicsDatasetBucketized
from utils.data_utils import get_omni_data,get_iaga_data_as_list
from utils.splitter import generate_indices
from utils.plot import SqueezedNorm
from Benchmark.metricgen import EventMetrics,Generate_metrics
from Benchmark.Forecaster import Forecaster,Generate_complete_weimer_forecast,Generate_weimer_forecast
from models.geoeffectivenet import *
import pickle
torch.set_default_dtype(torch.float64)  # this is important else it will overflow

In [ ]:
md = {'NeuralRNNWiemer_HidddenSuperMAG':NeuralRNNWiemer_HidddenSuperMAG,
        'NeuralRNNWiemer':NeuralRNNWiemer}

## Download and read data

The first step is to download the data, download the checkpoints, and download the code repository. The dataset contains two storm times -- 2011 storm and 2015 storm. These are standard storms which are used for evaluation of geomagnetic perturbation forecasting models.

The data, along with the model checkpoints are made available in a Google cloud bucket. The code has been made open source in a github repo.

In [ ]:
# Cell to download data from the bucket
!mkdir -p Storm
!gsutil cp -r gs://storm_subset/ Storm/

## Load hyperparameters

Hyperparameters are free parameters that are non-trainable, and  are fixed for a particular task. These values **must not** be changed during inference time, but we can make different realizations of the model with different values assigned to these hyperparameters.

In [ ]:
future_length = 1 #WAS DEFAULT IN THE CODE
past_omni_length = 120
nmax = 20
targets = ["dbe_nez", "dbn_nez"]
lag = 30
learning_rate = 5e-3
batch_size = 500
omni_resolution = 1
dropout_prob = 0.7
l2reg = 5e-5
n_hidden = 8
loss = 'MAE'
modname = "NeuralRNNWiemer"
basepath = "Storm/storm_subset/model/Best/"
# load scaler to unstandardize data
scalers = pickle.load(open(f"{basepath}scalers.p", "rb"))
_mean, _std = scalers["supermag"]

dbe_mean, dbn_mean = _mean
dbe_std, dbn_std = _std

### Load the data and the storm indices

As mentioned earlier, we have two datasets - years 2011 and 2015. Select any of the two years in the cell below, and run the notebook!

In [ ]:
YEAR = 2011
dataset = np.load(f"Storm/storm_subset/{YEAR}/supermag_omni_data.npz")
supermag_data = SuperMAGIAGADataset(dataset['dates'],dataset['data'],dataset['features'])
omni_data = OMNIDataset(pd.DataFrame(data=dataset['omni'],columns=dataset['omni_features']))
storm_idx = dataset['idx']

In [ ]:
storm_dataset = ShpericalHarmonicsDatasetBucketized(supermag_data,omni_data,storm_idx,
        f107_dataset=f"Storm/storm_subset/{YEAR}/f107.npz",targets=targets,past_omni_length=past_omni_length,
        past_supermag_length=1,future_length=future_length,lag=lag,zero_omni=False,
        zero_supermag=False,scaler=scalers,training_batch=False,nmax=nmax,inference=True)
storm_loader = data.DataLoader(storm_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
targets_idx = [np.where(storm_dataset.supermag_features == target)[0][0] for target in targets]

## Setup model

In [ ]:
NN_md = md[modname]
model = NN_md.load_from_checkpoint(f'{basepath}/epoch=8-step=3077.ckpt', strict=False, 
                                                 past_omni_length=past_omni_length,
                                                 future_length=future_length,
                                                 supermag_features=storm_dataset.supermag_features,
                                                 omni_features=storm_dataset.omni_features,
                                                 nmax=nmax,omni_resolution=omni_resolution,
                                                 targets_idx=targets_idx,learning_rate = learning_rate,
                                                 l2reg=l2reg,
                                                 dropout_prob=dropout_prob,
                                                 n_hidden=n_hidden,
                                                 loss=loss)

In [ ]:
if torch.cuda.is_available():
    model = model.cuda().double()
else:
    model = model.double()

## Generate forecast

In [ ]:
dbe_index = np.where(storm_dataset.supermag_features == 'dbe_nez')[0][0]
dbn_index = np.where(storm_dataset.supermag_features == 'dbn_nez')[0][0]
print(dbn_index,dbe_index)
target_index = {'dbe':dbe_index,'dbn':dbn_index}

In [ ]:
Predictions,Targets,All_times_coeff,Date_arr,MLT_sup_all,Mcolat_sup_all = Forecaster(storm_loader,model,
                                                                                     dbe_mean,dbe_std,dbn_mean,
                                                                                     dbn_std,target_index)

## Time series of forecast

In [ ]:
Date_arr['dbe'].shape,Predictions['dbe'].shape,Targets['dbe'].shape

### How does the forecast for one (any random) station look like?

In [ ]:
station_index = 100
plt.style.use('default')
fig,ax = plt.subplots(2,1,figsize=(12,8))

ax[0].plot(Predictions['dbe'][:,station_index],'orange',label='FDL')
ax[0].plot(Targets['dbe'][:,station_index],'k',label='Target')
ax[0].set_ylabel("$\\delta \mathrm{B}_{\mathrm{E}}$ (nT)")
ax[0].set_xlabel("Time from the start (min)")
ax[0].legend(loc='best')
ax[0].set_title(f"Forecast for station: {dataset['stations'][station_index]}",fontsize=20)

ax[1].plot(Predictions['dbn'][:,station_index],'orange',label='FDL')
ax[1].plot(Targets['dbn'][:,station_index],'k',label='Target')
ax[1].set_ylabel("$\\delta \mathrm{B}_{\mathrm{N}}$ (nT)")
ax[1].set_xlabel("Time from the start (min)")
ax[1].legend(loc='best')

**Figure 1**: A comparison of measured (black) and forecasted (orange) $\delta\mathrm{B}_{\mathrm{E}}$ (top) and $\delta\mathrm{B}_{\mathrm{N}}$ (bottom) for a given station.

-------------------

### How does the joint distribution of forecast and targets look like?


In [ ]:
plt.style.use('default')
fig,ax = plt.subplots(1,2,figsize=(12,6))

bad_indices = np.isnan(Targets['dbn']) | np.isnan(Predictions['dbn'])
im = ax[0].hist2d(Targets['dbn'][~bad_indices],Predictions['dbn'][~bad_indices],bins=150,cmap='cividis',norm=mpl.colors.LogNorm())
ax[0].set_ylabel("Predicted",fontsize=15)
ax[0].set_xlabel("Target",fontsize=15)
ax[0].set_xlim([np.nanmin(Predictions['dbn'].ravel()),np.nanmax(Targets['dbn'].ravel())+0.5])
ax[0].set_ylim(ax[0].get_xlim())
limy=ax[0].get_ylim()[0]#0.94
limx=ax[0].get_xlim()[-1]-0.1#0.99
ax[0].set_title("$\delta \mathrm{b}_{\mathrm{N}}$")
cbar = fig.colorbar(im[3], ax=ax[0], orientation='vertical')
cbar.ax.set_title("#",fontsize=15)
limy=ax[0].get_ylim()
limx=ax[0].get_xlim()
x_1 = np.linspace(limx[0],limx[1],4)
y_1 = x_1-limx[0]+limy[0]
ax[0].plot(x_1,y_1,'k')


bad_indices = np.isnan(Targets['dbe']) | np.isnan(Predictions['dbe'])
im  = ax[1].hist2d(Targets['dbe'][~bad_indices],Predictions['dbe'][~bad_indices],bins=150,cmap='cividis',norm=mpl.colors.LogNorm())
ax[1].set_ylabel("Predicted",fontsize=15)
ax[1].set_xlabel("Target",fontsize=15)
ax[1].set_xlim([np.nanmin(Predictions['dbe'].ravel()),np.nanpercentile(Targets['dbe'].ravel(),99.8)+0.5])
ax[1].set_ylim(ax[1].get_xlim())
limy=ax[1].get_ylim()[0]#0.94
limx=ax[1].get_xlim()[-1]-0.1#0.99
ax[1].set_title("$\delta \mathrm{b}_{\mathrm{E}}$")
cbar = fig.colorbar(im[3], ax=ax[1], orientation='vertical')
cbar.ax.set_title("#",fontsize=15)
limy=ax[1].get_ylim()
limx=ax[1].get_xlim()
x_1 = np.linspace(limx[0],limx[1],4)
y_1 = x_1-limx[0]+limy[0]
ax[1].plot(x_1,y_1,'k')

**Figure 2**: Joint histogram of measured and forecasted $\delta\mathrm{B}_{\mathrm{N}}$ (left) and $\delta\mathrm{B}_{\mathrm{E}}$ (right) for a given station.

-------------------

### Forecast for the top-3 best and worst performing stations!

In [ ]:
def plot_station(dt,fdl,targ,ax,**kwargs):
    l1 = ax.plot(dt,fdl,c="#00c5ff",label='FDL')
    ax.yaxis.label.set_color("#00c5ff")
    ax_tmp = ax.twinx()
    l2 = ax_tmp.plot(dt,targ,c="#181820",label=f"{kwargs['stat']}")
    ax.set_ylabel("$\\delta \mathrm{b}_{\mathrm{H}}$ (nT)",fontsize=15)
    ax_tmp.set_ylabel("$\\delta \mathrm{b}_{\mathrm{H}}$ (nT)",fontsize=15)
    ax.set_title(f"{kwargs['metric']} = {kwargs['mval']:.2f} nT")
    lns = l1+l2
    labs = [l.get_label() for l in lns]
    ax.spines['left'].set_color("#00c5ff")
    ax.legend(lns, labs, loc='best')
    ax.xaxis.get_majorticklabels()
    ax.tick_params(axis='x', rotation=-45)
    limy=ax.get_ylim()[-1]#0.94
    limx=ax.get_xlim()[0]-0.1#0.99
    

In [ ]:
dbh_pred.shape,dbh_targ.shape

In [ ]:
pl_dt = pd.to_datetime(Date_arr['dbe'].ravel(),unit='s')
symh_value = dataset['symh']
stations = dataset['stations']

dbh_pred = np.sqrt(Predictions['dbe']**2+Predictions['dbn']**2)
dbh_targ = np.sqrt(Targets['dbe']**2+Targets['dbn']**2)

subset = ~np.isnan(dbh_targ).any(axis=0)[:len(stations)]
dbh_pred = dbh_pred[:,:len(stations)][:,subset]
dbh_targ = dbh_targ[:,:len(stations)][:,subset]
stations = stations[subset]

mae = np.nanmean(np.abs(dbh_pred-dbh_targ),axis=0)
notnan = ~np.isnan(mae) 
sort_mae = np.argsort(mae[notnan])

mae = mae[notnan]
dbh_pred = dbh_pred[:,notnan]
dbh_targ = dbh_targ[:,notnan]
stations = stations[notnan]

i_bz = np.where(storm_dataset.omni_features=='bz')
i_s,i_e = storm_dataset.sg_indices[0,-1],storm_dataset.sg_indices[-1,-1]+1
bz = storm_dataset.omni[i_s:i_e,i_bz]*scalers['omni'][1][i_bz]+scalers['omni'][0][i_bz]

In [ ]:
plt.style.use('default')
fig,ax = plt.subplots(4,2,figsize=(12,12))

ax[0,0].plot(pl_dt,bz.ravel(),c="#181820")
ax[0,0].set_ylabel("IMF $\mathrm{b}_{\mathrm{z}}$ (nT)",fontsize=15)
ax[0,0].xaxis.get_majorticklabels()
ax[0,0].tick_params(axis='x', rotation=-45)
limy=ax[0,0].get_ylim()[-1]#0.94
limx=ax[0,0].get_xlim()[0]-0.1#0.99
    
ax[0,1].plot(pl_dt,symh_value.ravel(),c="#181820")
ax[0,1].set_ylabel("Sym-H (nT)",fontsize=15)
ax[0,1].xaxis.get_majorticklabels()
ax[0,1].tick_params(axis='x', rotation=-45)
limy=ax[0,1].get_ylim()[-1]#0.94
limx=ax[0,1].get_xlim()[0]-0.1#0.99

for i in np.arange(3):
    plot_station(pl_dt,dbh_pred[:,sort_mae[i]],dbh_targ[:,sort_mae[i]],ax[i+1,0],
                 metric='MAE',mval=mae[sort_mae[i]],pno=2*i,stat=stations[sort_mae[i]])
for i in np.arange(3):
    plot_station(pl_dt,dbh_pred[:,sort_mae[-i-1]],dbh_targ[:,sort_mae[-i-1]],ax[i+1,1],
                 metric='MAE',mval=mae[sort_mae[-i-1]],pno=2*i+1,stat=stations[sort_mae[-i-1]])
fig.suptitle(f"{YEAR} storm: Comparison of performance for the top - 3 best and worst performing stations",fontsize=15)
fig.tight_layout()

**Figure 3**: The IMF Bz (left), Sym-H (right) in the first rwo, and top 3 best (left column) and worst (right columns) performing stations in the subsequent rows. The blue colour indicates forecast from our model, while the black colour indicates measurements at different stations (in the legend of each figure), with the MAE reported on top.

-------------------

## Forecast all over the Earth

### Generate latitude-longitude grid

In [ ]:
MLT_sup_all.keys()

In [ ]:
_mlt = np.linspace(np.nanmin(MLT_sup_all['dbe']),np.nanmax(MLT_sup_all['dbe']),100)
_colat = np.linspace(np.nanmin(Mcolat_sup_all['dbe']),np.nanmax(Mcolat_sup_all['dbe']),360)
_mlt,_colat = np.meshgrid(_mlt,_colat)
_basis = basis_matrix(nmax ,_mlt, _colat)


### Transform coefficients to forecast

In [ ]:
fdl_on_grid_gif = {'dbe':[],'dbn':[]}

fdl_on_grid_gif['dbn'] = np.einsum('bij,lj->bil',_basis,All_times_coeff['dbn'])
fdl_on_grid_gif['dbe'] = np.einsum('bij,lj->bil',_basis,All_times_coeff['dbe'])

fdl_on_grid_gif['dbn'] = fdl_on_grid_gif['dbn']*dbn_std + dbn_mean
fdl_on_grid_gif['dbe'] = fdl_on_grid_gif['dbe']*dbe_std + dbe_mean

In [ ]:
# Sanity check.
print(fdl_on_grid_gif['dbn'].shape,fdl_on_grid_gif['dbe'].shape)

## What does the global forecast look like?

We can plot the forecast and measurements across the globe ==> get global maps!

In [ ]:
plt.style.use("dark_background")

In [ ]:
k = 'dbn'
f = 1
fig1 = plt.figure(figsize=(12*f,8*f),dpi=100)
gs = fig1.add_gridspec(8, 2)

theta = _mlt 
phi = _colat 
cmap=cm.get_cmap("RdYlBu_r",17)
ax1 = fig1.add_subplot(gs[:-1,0], projection='polar')
ax1.set_theta_offset(-np.pi/2)
ax2 = fig1.add_subplot(gs[:-1,1], projection='polar')
ax2.set_theta_offset(-np.pi/2)
# ax3 = fig1.add_subplot(gs[:-1,2], projection='polar')
# ax3.set_theta_offset(-np.pi/2)
cax = fig1.add_subplot(gs[-1,:])
time_ind=1500
d = storm_dataset.dates[time_ind]
cax.cla()
dt = pd.to_datetime(d,unit='s')

dbH2 = fdl_on_grid_gif[k][...,time_ind] #np.sqrt(np.square(Targets['dbe'][:,:,start:end+1])+np.square(Targets['dbn'][:,:,start:end+1]))

maxval = np.nanpercentile(dbH2,95)
minval = np.nanpercentile(dbH2,5)
norm=SqueezedNorm(vmin=minval, vmax=maxval, mid=np.nanmean(dbH2), s1=2, s2=2)

dbH = fdl_on_grid_gif[k][...,time_ind] #np.sqrt(np.square(fdl_on_weimergrid_gif['dbe'][:,:,start:end+1][...,time_ind])+np.square(fdl_on_weimergrid_gif['dbn'][:,:,start:end+1][...,time_ind]))
c2=ax2.pcolormesh(theta,phi, dbH,cmap=cmap,norm=norm)  
ax2.set_ylim([np.nanmin(Mcolat_sup_all[k][time_ind]),np.nanmax(Mcolat_sup_all[k][time_ind])])
ax2.grid(linewidth=1,color='white',linestyle='--')
ax2.text(3.5,1.1,f"FDL 30-min forecast",fontsize=15*f)

dbH = Targets[k][time_ind] #np.sqrt(np.square(Targets['dbe'][start:end+1][time_ind])+np.square(Targets['dbn'][start:end+1][time_ind]))
c3=ax1.scatter(MLT_sup_all[k][time_ind],Mcolat_sup_all[k][time_ind], c=dbH,cmap=cmap,norm=norm,s=36*f)
ax1.text(3.45,1.16,f"Target SuperMag",fontsize=15*f)
ax1.grid(linewidth=2,color='white')
fig1.colorbar(c3, cax=cax,orientation='horizontal')
cax.set_xlabel(r'Northward magnetic field perturbation ($\delta\mathrm{B}_{\mathrm{N}}$) (nT)',fontsize=15)


**Figure 4**: Global maps of measurement (left) and forecast (right).

-------------------

In [ ]:
from celluloid import Camera

In [ ]:
k = 'dbn'
f = 1
fig1 = plt.figure(figsize=(12*f,8*f),dpi=100)
gs = fig1.add_gridspec(8, 2)

camera = Camera(fig1)
theta = _mlt 
phi = _colat 
cmap=cm.get_cmap("RdYlBu_r",17)
ax1 = fig1.add_subplot(gs[:-1,0], projection='polar')
ax1.set_theta_offset(-np.pi/2)
ax2 = fig1.add_subplot(gs[:-1,1], projection='polar')
ax2.set_theta_offset(-np.pi/2)
# ax3 = fig1.add_subplot(gs[:-1,2], projection='polar')
# ax3.set_theta_offset(-np.pi/2)
cax = fig1.add_subplot(gs[-1,:])

for time_ind,d in enumerate(storm_dataset.dates[1500:][::12]):
    cax.cla()
    dt = pd.to_datetime(d,unit='s')
    
    dbH2 = fdl_on_grid_gif[k][...,time_ind] #np.sqrt(np.square(Targets['dbe'][:,:,start:end+1])+np.square(Targets['dbn'][:,:,start:end+1]))

    maxval = np.nanpercentile(dbH2,95)
    minval = np.nanpercentile(dbH2,5)
    norm=SqueezedNorm(vmin=minval, vmax=maxval, mid=np.nanmean(dbH2), s1=2, s2=2)

    dbH = fdl_on_grid_gif[k][...,time_ind] #np.sqrt(np.square(fdl_on_weimergrid_gif['dbe'][:,:,start:end+1][...,time_ind])+np.square(fdl_on_weimergrid_gif['dbn'][:,:,start:end+1][...,time_ind]))
    c2=ax2.pcolormesh(theta,phi, dbH,cmap=cmap,norm=norm)  
    ax2.set_ylim([np.nanmin(Mcolat_sup_all[k][time_ind]),np.nanmax(Mcolat_sup_all[k][time_ind])])
    ax2.grid(linewidth=1,color='white',linestyle='--')
    ax2.text(3.5,1.1,f"FDL 30-min forecast",fontsize=15*f)

    dbH = Targets[k][time_ind] #np.sqrt(np.square(Targets['dbe'][start:end+1][time_ind])+np.square(Targets['dbn'][start:end+1][time_ind]))
    c3=ax1.scatter(MLT_sup_all[k][time_ind],Mcolat_sup_all[k][time_ind], c=dbH,cmap=cmap,norm=norm,s=36*f)
    ax1.text(3.45,1.16,f"Target SuperMag",fontsize=15*f)
    ax1.grid(linewidth=2,color='white')
    fig1.colorbar(c3, cax=cax,orientation='horizontal')
    cax.set_xlabel(r'Northward magnetic field perturbation ($\delta\mathrm{B}_{\mathrm{N}}$) (nT)',fontsize=15)
    camera.snap()

**Figure 5**: Global maps of $\delta\mathrm{B}_{\mathrm{N}}$measurement (left) and forecast (right).

-------------------

#### If you prefer saving the animation as a video, use `animation.save()`. Otherwise, run the other cell!

In [ ]:
animation = camera.animate()

In [ ]:
#animation.save('dbn.mp4',bitrate=-1)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

In [ ]:
k = 'dbe'
f = 1
fig1 = plt.figure(figsize=(12*f,8*f),dpi=100)
gs = fig1.add_gridspec(8, 2)

camera = Camera(fig1)
theta = _mlt 
phi = _colat 
cmap=cm.get_cmap("RdYlBu_r",17)
ax1 = fig1.add_subplot(gs[:-1,0], projection='polar')
ax1.set_theta_offset(-np.pi/2)
ax2 = fig1.add_subplot(gs[:-1,1], projection='polar')
ax2.set_theta_offset(-np.pi/2)
# ax3 = fig1.add_subplot(gs[:-1,2], projection='polar')
# ax3.set_theta_offset(-np.pi/2)
cax = fig1.add_subplot(gs[-1,:])

for time_ind,d in enumerate(storm_dataset.dates[1500:][::12]):
    cax.cla()
    dt = pd.to_datetime(d,unit='s')
    
    dbH2 = fdl_on_grid_gif[k][...,time_ind] #np.sqrt(np.square(Targets['dbe'][:,:,start:end+1])+np.square(Targets['dbn'][:,:,start:end+1]))

    maxval = np.nanpercentile(dbH2,95)
    minval = np.nanpercentile(dbH2,5)
    norm=SqueezedNorm(vmin=minval, vmax=maxval, mid=np.nanmean(dbH2), s1=3, s2=3)

    dbH = fdl_on_grid_gif[k][...,time_ind] #np.sqrt(np.square(fdl_on_weimergrid_gif['dbe'][:,:,start:end+1][...,time_ind])+np.square(fdl_on_weimergrid_gif['dbn'][:,:,start:end+1][...,time_ind]))
    c2=ax2.pcolormesh(theta,phi, dbH,cmap=cmap,norm=norm)  
    ax2.set_ylim([np.nanmin(Mcolat_sup_all[k][time_ind]),np.nanmax(Mcolat_sup_all[k][time_ind])])
    ax2.grid(linewidth=1,color='white',linestyle='--')
    ax2.text(3.5,1.1,f"FDL 30-min forecast",fontsize=15*f)

    dbH = Targets[k][time_ind] #np.sqrt(np.square(Targets['dbe'][start:end+1][time_ind])+np.square(Targets['dbn'][start:end+1][time_ind]))
    c3=ax1.scatter(MLT_sup_all[k][time_ind],Mcolat_sup_all[k][time_ind], c=dbH,cmap=cmap,norm=norm,s=36*f)
    ax1.text(3.45,1.16,f"Target SuperMag",fontsize=15*f)
    ax1.grid(linewidth=2,color='white')
    fig1.colorbar(c3, cax=cax,orientation='horizontal')
    cax.set_xlabel(r'Eastward magnetic field perturbation ($\delta\mathrm{B}_{\mathrm{E}}$) (nT)',fontsize=15)
    camera.snap()

**Figure 6**: Global maps of $\delta\mathrm{B}_{\mathrm{E}}$measurement (left) and forecast (right).

-------------------

#### If you prefer saving the animation as a video, use `animation.save()`. Otherwise, run the other cell!

In [ ]:
animation = camera.animate()

In [ ]:
# animation.save('dbe.mp4',bitrate=-1)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())